In [ ]:
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

from pymedphys.msq import mosaiq_connect, delivery_data_from_mosaiq, get_patient_fields
from pymedphys.mudensity import calc_mu_density
from pymedphys.logfile import get_delivery_parameters

from pymedphys.trf import delivery_data_from_logfile

# from analyse_logfiles import get_delivery_parameters

In [ ]:
clinical_server = 'physics-server:31433'  # nbcc
qa_server = 'msqsql'  # rccc

# username = 'physics'

In [ ]:
patient_id = '--'

with mosaiq_connect(clinical_server) as cursor:
    fields = get_patient_fields(cursor, patient_id)
    
fields

In [ ]:
clinical_beam_reference = (fields['field_version'] == 0) & (fields['monitor_units'] != 0)
clinical_fields = fields[clinical_beam_reference]
clinical_fields

In [ ]:
qa_patient_id = '999994'

with mosaiq_connect(qa_server) as cursor:
    qa_fields = get_patient_fields(cursor, qa_patient_id)
    
site_with_patient_id = qa_fields[qa_fields['site'] == patient_id]
site_with_patient_id

In [ ]:
clinical_field_ids = list(clinical_fields['field_id'].values.astype(str))
qa_field_ids = list(site_with_patient_id['field_id'].values.astype(str))

In [ ]:
qa_field_ids

In [ ]:
def determine_mu_density(server, field_id):
    with mosaiq_connect(server) as cursor:
        delivery_data = delivery_data_from_mosaiq(cursor, field_id)
        
    mu, mlc, jaw = get_delivery_parameters(delivery_data)
    mu_density = calc_mu_density(mu, mlc, jaw)
    
    return mu_density

In [ ]:
clinical_mu_density = np.sum([
    determine_mu_density(clinical_server, field_id)
    for field_id in clinical_field_ids
], axis=0)

In [ ]:
qa_mu_density = np.sum([
    determine_mu_density(qa_server, field_id)
    for field_id in qa_field_ids
], axis=0)

In [ ]:
plt.figure()
plt.pcolormesh(clinical_mu_density)
plt.colorbar()
plt.title('Clinical MU density')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')
plt.gca().invert_yaxis()

plt.figure()
plt.pcolormesh(qa_mu_density)
plt.colorbar()
plt.title('QA MU density')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')
plt.gca().invert_yaxis()

In [ ]:
# Verify that every pixel agrees within 0.1 MU
np.all(np.abs(clinical_mu_density - qa_mu_density) < 0.1)